In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Image,display
from langgraph.graph import StateGraph,START,END,add_messages
from langchain_core.messages import AnyMessage,HumanMessage,AIMessage
from langgraph.prebuilt import ToolNode,tools_condition
from typing_extensions import TypedDict,Annotated
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel,Field

In [2]:
_ = load_dotenv()
llm = ChatGroq(model="qwen-2.5-32b")
result = llm.invoke("what is capital of india")
result

AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 34, 'total_tokens': 43, 'completion_time': 0.045, 'prompt_time': 0.003910996, 'queue_time': 0.225250049, 'total_time': 0.048910996}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'stop', 'logprobs': None}, id='run-0cc0c2c1-5435-48c9-b58f-eb72e853f96c-0', usage_metadata={'input_tokens': 34, 'output_tokens': 9, 'total_tokens': 43})

In [6]:
class MessageState(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [ ]:
class grade(BaseModel):
        """Binary score for relevance check."""

        binary_score: str = Field(description="Relevance score 'yes' or 'no'")

model = ChatGroq(model="qwen-2.5-32b")

# LLM with tool and validation
llm_with_tool = model.with_structured_output(grade)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {context} \n\n
    Here is the user question: {question} \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.""",
    input_variables=["context", "question"],
)

# Chain
chain = prompt | llm_with_tool